**Problem Statement**: This notebook provides the details about Machine learning predictive model for the problem “Bank loan default risk”.

# **1. importing required libraries and reading the data**


In [1]:
# Code to mount google drive   (loading the data from your google drive)
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold      # for Feature selection
from sklearn.feature_selection import mutual_info_classif    # Feature selection - mutual information
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from sklearn.ensemble import IsolationForest



# **Reading data from pickle file**

In [9]:
import pickle
import joblib 
import pandas as pd

# joblib.dump(FinalXGboostModel, r'/gdrive/MyDrive/PGD_project/FinalXGboostModel.pkl') 
# model = joblib.load(r'/gdrive/MyDrive/PGD_project/FinalXGboostModel.pkl') 
# model.predict(X_test)

DataForML_t=pd.read_pickle(r'/gdrive/MyDrive/PGD_project/DataForML_t.pkl')

In [10]:
DataForML_t.head()

,TARGET,AGE,AMT_CREDIT,AMT_INCOME_TOTAL,CNT_FAM_MEMBERS,YEARS_EMPLOYED,AMT_ANNUITY,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_2,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,CODE_GENDER,FLAG_OWN_CAR,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_EDUCATION_TYPE,NAME_CONTRACT_TYPE,FLAG_OWN_REALTY,NAME_TYPE_SUITE,ORGANIZATION_TYPE
0,1,25.0,406597.5,202500.0,1.0,1.0,24700.5,1,0,1,1,0,2,2,0,0,0.262949,1,3648.0,2120,2,2.0,Male,N,Working,Single / not married,House / apartment,Secondary / secondary special,Cash loans,Y,Unaccompanied,Business Entity Type 3
1,0,45.0,1293502.5,270000.0,2.0,3.0,35698.5,1,0,1,1,0,1,1,0,0,0.622246,0,1186.0,291,1,0.0,Female,N,State servant,Married,House / apartment,Higher education,Cash loans,N,Family,School
2,0,52.0,135000.0,67500.0,1.0,0.0,6750.0,1,1,1,0,0,2,2,0,0,0.555912,0,4260.0,2531,0,0.0,Male,Y,Working,Single / not married,House / apartment,Secondary / secondary special,Revolving loans,Y,Unaccompanied,Government
3,0,52.0,312682.5,135000.0,2.0,8.0,29686.5,1,0,0,1,0,2,2,0,0,0.650442,0,9833.0,2437,2,0.0,Female,N,Working,Civil marriage,House / apartment,Secondary / secondary special,Cash loans,Y,Unaccompanied,Business Entity Type 3
4,0,54.0,513000.0,121500.0,1.0,8.0,21865.5,1,0,0,0,0,2,2,0,1,0.322738,0,4311.0,3458,0,0.0,Male,N,Working,Single / not married,House / apartment,Secondary / secondary special,Cash loans,Y,Unaccompanied,Religion


In [11]:
DataForML_t['CODE_GENDER'].replace({'Male':1, 'Female':0}, inplace=True)
DataForML_t['FLAG_OWN_CAR'].replace({'Y':1, 'N':0}, inplace=True)
DataForML_t['NAME_CONTRACT_TYPE'].replace({'Revolving loans':1, 'Cash loans':0}, inplace=True)
DataForML_t['FLAG_OWN_REALTY'].replace({'Y':1, 'N':0}, inplace=True)

In [12]:
# Treating all the nominal variables at once using dummy variables
DataForML_Numeric=pd.get_dummies(DataForML_t)
DataForML_Numeric.head()

,TARGET,AGE,AMT_CREDIT,AMT_INCOME_TOTAL,CNT_FAM_MEMBERS,YEARS_EMPLOYED,AMT_ANNUITY,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_2,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,CODE_GENDER,FLAG_OWN_CAR,NAME_CONTRACT_TYPE,FLAG_OWN_REALTY,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,...,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA
0,1,25.0,406597.5,202500.0,1.0,1.0,24700.5,1,0,1,1,0,2,2,0,0,0.262949,1,3648.0,2120,2,2.0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,45.0,1293502.5,270000.0,2.0,3.0,35698.5,1,0,1,1,0,1,1,0,0,0.622246,0,1186.0,291,1,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,52.0,135000.0,67500.0,1.0,0.0,6750.0,1,1,1,0,0,2,2,0,0,0.555912,0,4260.0,2531,0,0.0,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,52.0,312682.5,135000.0,2.0,8.0,29686.5,1,0,0,1,0,2,2,0,0,0.650442,0,9833.0,2437,2,0.0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,54.0,513000.0,121500.0,1.0,8.0,21865.5,1,0,0,0,0,2,2,0,1,0.322738,0,4311.0,3458,0,0.0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
print(DataForML_Numeric.shape)

(306486, 116)


# **Machine Learning**



In [14]:
# Printing all the column names for our reference
print(list(DataForML_Numeric.columns))

['TARGET', 'AGE', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'CNT_FAM_MEMBERS', 'YEARS_EMPLOYED', 'AMT_ANNUITY', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_6', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'EXT_SOURCE_2', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE_Businessman', 'NAME_INCOME_TYPE_Commercial associate', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_INCOME_TYPE_Pensioner', 'NAME_INCOME_TYPE_State servant', 'NAME_INCOME_TYPE_Student', 'NAME_INCOME_TYPE_Unemployed', 'NAME_INCOME_TYPE_Working', 'NAME_FAMILY_STATUS_Civil marriage', 'NAME_FAMILY_STATUS_Married', 'NAME_FAMILY_STATUS_Separated', 'NAME_FAMILY_STATUS_Single / not married', 'NAME_FAMILY_STATUS_Unknown', 'NAME_FAMILY_STATUS_Widow', 'NAME_HOUSING_TYPE_Co-op a

In [15]:
# Separate Target Variable and Predictor Variables
TargetVariable='TARGET'
Predictors=[ 'AGE', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'CNT_FAM_MEMBERS', 'YEARS_EMPLOYED', 'AMT_ANNUITY', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 
             'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_6', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 
             'EXT_SOURCE_2', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 
             'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',  'FLAG_OWN_REALTY',
             
             'NAME_TYPE_SUITE_Unaccompanied', 'NAME_TYPE_SUITE_Family', 'NAME_TYPE_SUITE_Spouse, partner', 'NAME_TYPE_SUITE_Children', 
             'NAME_TYPE_SUITE_Other_B', 'NAME_TYPE_SUITE_Other_A', 'NAME_TYPE_SUITE_Group of people',
             
             'NAME_EDUCATION_TYPE_Secondary / secondary special', 'NAME_EDUCATION_TYPE_Higher education', 'NAME_EDUCATION_TYPE_Incomplete higher',
             'NAME_EDUCATION_TYPE_Lower secondary', 'NAME_EDUCATION_TYPE_Academic degree',  
               
             'NAME_FAMILY_STATUS_Married', 'NAME_FAMILY_STATUS_Single / not married', 'NAME_FAMILY_STATUS_Civil marriage', 
             'NAME_FAMILY_STATUS_Separated', 'NAME_FAMILY_STATUS_Widow', 'NAME_FAMILY_STATUS_Unknown',     

             'NAME_HOUSING_TYPE_House / apartment', 'NAME_HOUSING_TYPE_With parents', 'NAME_HOUSING_TYPE_Municipal apartment',  
             'NAME_HOUSING_TYPE_Rented apartment', 'NAME_HOUSING_TYPE_Office apartment', 'NAME_HOUSING_TYPE_Co-op apartment',         

             'ORGANIZATION_TYPE_Business Entity Type 3', 'ORGANIZATION_TYPE_XNA', 'ORGANIZATION_TYPE_Self-employed', 'ORGANIZATION_TYPE_Other',
             'ORGANIZATION_TYPE_Medicine', 'ORGANIZATION_TYPE_Business Entity Type 2', 'ORGANIZATION_TYPE_Government', 'ORGANIZATION_TYPE_School',
             'ORGANIZATION_TYPE_Trade: type 7', 'ORGANIZATION_TYPE_Kindergarten', 'ORGANIZATION_TYPE_Construction', 'ORGANIZATION_TYPE_Business Entity Type 1',
             'ORGANIZATION_TYPE_Transport: type 4', 'ORGANIZATION_TYPE_Trade: type 3', 'ORGANIZATION_TYPE_Industry: type 9', 'ORGANIZATION_TYPE_Industry: type 3',
             'ORGANIZATION_TYPE_Security', 'ORGANIZATION_TYPE_Housing', 'ORGANIZATION_TYPE_Industry: type 11', 'ORGANIZATION_TYPE_Military',
             'ORGANIZATION_TYPE_Bank', 'ORGANIZATION_TYPE_Agriculture', 'ORGANIZATION_TYPE_Police', 'ORGANIZATION_TYPE_Transport: type 2',
             'ORGANIZATION_TYPE_Postal', 'ORGANIZATION_TYPE_Security Ministries', 'ORGANIZATION_TYPE_Trade: type 2', 'ORGANIZATION_TYPE_Restaurant',
             'ORGANIZATION_TYPE_Services', 'ORGANIZATION_TYPE_University', 'ORGANIZATION_TYPE_Industry: type 7', 'ORGANIZATION_TYPE_Transport: type 3',
             'ORGANIZATION_TYPE_Industry: type 1', 'ORGANIZATION_TYPE_Hotel', 'ORGANIZATION_TYPE_Electricity', 'ORGANIZATION_TYPE_Industry: type 4',
             'ORGANIZATION_TYPE_Trade: type 6', 'ORGANIZATION_TYPE_Industry: type 5', 'ORGANIZATION_TYPE_Insurance', 'ORGANIZATION_TYPE_Telecom',
             'ORGANIZATION_TYPE_Emergency', 'ORGANIZATION_TYPE_Industry: type 2', 'ORGANIZATION_TYPE_Advertising', 'ORGANIZATION_TYPE_Realtor',
             'ORGANIZATION_TYPE_Culture', 'ORGANIZATION_TYPE_Industry: type 12', 'ORGANIZATION_TYPE_Trade: type 1',
             'ORGANIZATION_TYPE_Mobile', 'ORGANIZATION_TYPE_Legal Services', 'ORGANIZATION_TYPE_Cleaning', 'ORGANIZATION_TYPE_Transport: type 1', 
             'ORGANIZATION_TYPE_Industry: type 6', 'ORGANIZATION_TYPE_Industry: type 10', 'ORGANIZATION_TYPE_Religion', 'ORGANIZATION_TYPE_Industry: type 13',
             'ORGANIZATION_TYPE_Trade: type 4', 'ORGANIZATION_TYPE_Trade: type 5', 'ORGANIZATION_TYPE_Industry: type 8',    
                  
             'NAME_INCOME_TYPE_Working', 'NAME_INCOME_TYPE_Commercial associate', 'NAME_INCOME_TYPE_Pensioner', 'NAME_INCOME_TYPE_State servant', 
             'NAME_INCOME_TYPE_Unemployed', 'NAME_INCOME_TYPE_Student', 'NAME_INCOME_TYPE_Businessman', 'NAME_INCOME_TYPE_Maternity leave']

X_ml=DataForML_Numeric[Predictors].values
y_ml=DataForML_Numeric[TargetVariable].values

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(X_ml, y_ml, test_size=0.3, random_state=428)


# **6B. Standardization/Normalization of data**

In [ ]:
### Sandardization of data ###
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Choose either standardization or Normalization
# On this data Min Max Normalization produced better results

# Choose between standardization and MinMAx normalization
#PredictorScaler=StandardScaler()
PredictorScaler=MinMaxScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X_ml)

# Generating the standardized values of X
X_ml=PredictorScalerFit.transform(X_ml)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(X_ml, y_ml, test_size=0.3, random_state=42)

In [ ]:
# Saving this standardizer for deployment
import pickle
# DataForML.to_pickle('DataForML.pkl')
PredictorScalerFit.to_pickle(r'/gdrive/MyDrive/PGD_project/PredictorScalerFit.pkl')

In [ ]:
# Sanity check for the sampled data
print(X_train_ml.shape)
print(y_train_ml.shape)
print(X_test_ml.shape)
print(y_test_ml.shape)

(214540, 115)
(214540,)
(91946, 115)
(91946,)


In [ ]:
# Get the default and non-default

# default_1     = y_test[y_test['TARGET']==1]
# non_default_1 = y_test[y_test['TARGET']==0]

default_1     = y_test_ml[y_test_ml==1]
non_default_1 = y_test_ml[y_test_ml==0]

print(default_1.shape,non_default_1.shape)

(7437,) (84509,)


# **Oversampling**

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
os = RandomOverSampler(0.75)
X_train_mlos, y_train_mlos = os.fit_resample(X_train_ml, y_train_ml)

print("The number of classes before fit {}".format(Counter(y_train_ml)))
print("The number of classes after fit {}".format(Counter(y_train_mlos)))

/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:591: FutureWarning: Pass sampling_strategy=0.75 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning,


The number of classes before fit Counter({0: 197188, 1: 17352})
The number of classes after fit Counter({0: 197188, 1: 147891})


In [ ]:
X_train_mlos.shape

(345079, 115)

In [ ]:
X_train_ml.shape

(214540, 115)

# **XGBoost(Oversampling)**

In [ ]:
# Xtreme Gradient Boosting (XGBoost)
from xgboost import XGBClassifier
clf=XGBClassifier(max_depth=7, learning_rate=0.1, n_estimators=1000, objective='binary:logistic', booster='gbtree')

# Printing all the parameters of XGBoost
print(clf)

# Creating the model on Training Data
XGB=clf.fit(X_train_mlos,y_train_mlos)
prediction=XGB.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction))
print(metrics.confusion_matrix(y_test_ml, prediction))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

XGBClassifier(max_depth=7, n_estimators=1000)
              precision    recall  f1-score   support

           0       0.94      0.88      0.91     84509
           1       0.20      0.33      0.25      7437

    accuracy                           0.84     91946
   macro avg       0.57      0.61      0.58     91946
weighted avg       0.88      0.84      0.86     91946

[[74498 10011]
 [ 4960  2477]]
Accuracy of the model on Testing Sample Data: 0.86


In [ ]:
import joblib 
joblib.dump(XGB, r'/gdrive/MyDrive/PGD_project/FinalXGboostModel_os0_171000.pkl') 
model = joblib.load(r'/gdrive/MyDrive/PGD_project/FinalXGboostModel_os0_171000.pkl') 

# **Logistic Regression(oversampling)**

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# choose parameter Penalty='l1' or C=1
# choose different values for solver 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'
clf = LogisticRegression(C=1,penalty='l2', solver='newton-cg')

# Printing all the parameters of logistic regression
# print(clf)

# Creating the model on Training Data
LOG=clf.fit(X_train_mlos,y_train_mlos)
prediction_lr=LOG.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_lr))
print(metrics.confusion_matrix(y_test_ml, prediction_lr))

# Printing the Overall Accuracy of the model
F1_Score_lr=metrics.f1_score(y_test_ml, prediction_lr, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score_lr,2))

# # Importing cross validation function from sklearn
# from sklearn.model_selection import cross_val_score

# # Running 10-Fold Cross validation on a given algorithm
# # Passing full data X and y because the K-fold will split the data and automatically choose train/test
# Accuracy_Values=cross_val_score(LOG, X , y, cv=10, scoring='f1_weighted')
# print('\nAccuracy values for 10-fold Cross Validation:\n',Accuracy_Values)
# print('\nFinal Average Accuracy of the model:', round(Accuracy_Values.mean(),2))

              precision    recall  f1-score   support

           0       0.95      0.79      0.86     84509
           1       0.17      0.49      0.25      7437

    accuracy                           0.76     91946
   macro avg       0.56      0.64      0.55     91946
weighted avg       0.88      0.76      0.81     91946

[[66425 18084]
 [ 3779  3658]]
Accuracy of the model on Testing Sample Data: 0.81


In [ ]:
# example of grid searching key hyperparametres for logistic regression
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# choose parameter Penalty='l1' or C=1
# choose different values for solver 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'
clf = LogisticRegression(C=100,penalty='l2', solver='liblinear')

# Printing all the parameters of logistic regression
# print(clf)

# Creating the model on Training Data
LOG=clf.fit(X_train_mlos,y_train_mlos)
prediction_lr=LOG.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_lr))
print(metrics.confusion_matrix(y_test_ml, prediction_lr))

# Printing the Overall Accuracy of the model
F1_Score_lr=metrics.f1_score(y_test_ml, prediction_lr, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score_lr,2))

              precision    recall  f1-score   support

           0       0.95      0.78      0.86     84509
           1       0.17      0.50      0.25      7437

    accuracy                           0.76     91946
   macro avg       0.56      0.64      0.55     91946
weighted avg       0.88      0.76      0.81     91946

[[66230 18279]
 [ 3746  3691]]
Accuracy of the model on Testing Sample Data: 0.81


# **KNN(Oversampling)**

In [ ]:
# K-Nearest Neighbor(KNN)
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)

# Printing all the parameters of KNN
print(clf)

# Creating the model on Training Data
KNN=clf.fit(X_train_mlos,y_train_mlos)
prediction_knn=KNN.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_knn))
print(metrics.confusion_matrix(y_test_ml, prediction_knn))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_knn, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

KNeighborsClassifier()
              precision    recall  f1-score   support

           0       0.93      0.80      0.86     84509
           1       0.11      0.28      0.16      7437

    accuracy                           0.75     91946
   macro avg       0.52      0.54      0.51     91946
weighted avg       0.86      0.75      0.80     91946

[[67318 17191]
 [ 5361  2076]]
Accuracy of the model on Testing Sample Data: 0.8


# **DecisionTree(Oversampling)**

In [ ]:
#Decision Trees
from sklearn import tree
#choose from different tunable hyper parameters
clf = tree.DecisionTreeClassifier(max_depth=7,criterion='entropy')

# Printing all the parameters of Decision Trees
print(clf)

# Creating the model on Training Data
DTree=clf.fit(X_train_mlos,y_train_mlos)
prediction_dt=DTree.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_dt))
print(metrics.confusion_matrix(y_test_ml, prediction_dt))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_dt, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

DecisionTreeClassifier(criterion='entropy', max_depth=7)
              precision    recall  f1-score   support

           0       0.94      0.76      0.84     84509
           1       0.15      0.50      0.23      7437

    accuracy                           0.74     91946
   macro avg       0.55      0.63      0.54     91946
weighted avg       0.88      0.74      0.79     91946

[[63895 20614]
 [ 3743  3694]]
Accuracy of the model on Testing Sample Data: 0.79


# **Randomforest(oversampling)**

Final RF

In [ ]:
# Random Forest (Bagging of multiple Decision Trees)
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=3, n_estimators=100,criterion='gini')

# Printing all the parameters of Random Forest
print(clf)

# Creating the model on Training Data
RF=clf.fit(X_train_mlos,y_train_mlos)
prediction_rf=RF.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_rf))
print(metrics.confusion_matrix(y_test_ml, prediction_rf))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_rf, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

RandomForestClassifier(max_depth=3)
              precision    recall  f1-score   support

           0       0.93      0.95      0.94     84509
           1       0.23      0.18      0.20      7437

    accuracy                           0.89     91946
   macro avg       0.58      0.56      0.57     91946
weighted avg       0.87      0.89      0.88     91946

[[80078  4431]
 [ 6130  1307]]
Accuracy of the model on Testing Sample Data: 0.88


# **Adaboost**

In [ ]:
# Adaboost 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Choosing Decision Tree with 1 level as the weak learner
DTC=DecisionTreeClassifier(max_depth=1)
clf = AdaBoostClassifier(n_estimators=500, base_estimator=DTC ,learning_rate=0.01)

# Printing all the parameters of Adaboost
print(clf)

# Creating the model on Training Data
AB=clf.fit(X_train_mlos,y_train_mlos)
prediction_ab=AB.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_ab))
print(metrics.confusion_matrix(y_test_ml, prediction_ab))

# Printing the Overall Accuracy of the model
F1_Score_ab=metrics.f1_score(y_test_ml, prediction_ab, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score_ab,2))

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.01, n_estimators=500)
              precision    recall  f1-score   support

           0       0.94      0.83      0.88     84509
           1       0.17      0.40      0.24      7437

    accuracy                           0.79     91946
   macro avg       0.56      0.62      0.56     91946
weighted avg       0.88      0.79      0.83     91946

[[70035 14474]
 [ 4440  2997]]
Accuracy of the model on Testing Sample Data: 0.83


# **Naive Bayes(Oversampling)**

In [ ]:

# Naive Bays
from sklearn.naive_bayes import GaussianNB, MultinomialNB

# GaussianNB is used in Binomial Classification
# MultinomialNB is used in multi-class classification
clf = GaussianNB()
#clf = MultinomialNB()

# Printing all the parameters of Naive Bayes
print(clf)

NB=clf.fit(X_train_mlos,y_train_mlos)
# NB=clf.fit(X_train_ml,y_train_ml)
# NB=clf.fit(X_ml,y_ml)


prediction_nb=NB.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_nb))
print(metrics.confusion_matrix(y_test_ml, prediction_nb))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_nb, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

GaussianNB()
              precision    recall  f1-score   support

           0       0.95      0.06      0.12     84509
           1       0.08      0.96      0.15      7437

    accuracy                           0.13     91946
   macro avg       0.51      0.51      0.13     91946
weighted avg       0.88      0.13      0.12     91946

[[ 5268 79241]
 [  303  7134]]
Accuracy of the model on Testing Sample Data: 0.12


In [ ]:
# Naive Bays
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

# GaussianNB is used in Binomial Classification
# MultinomialNB is used in multi-class classification
clf = BernoulliNB()
#clf = MultinomialNB()

# Printing all the parameters of Naive Bayes
print(clf)

NB=clf.fit(X_train_mlos,y_train_mlos)
# NB=clf.fit(X_train_ml,y_train_ml)
# NB=clf.fit(X_ml,y_ml)


prediction_nb=NB.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_nb))
print(metrics.confusion_matrix(y_test_ml, prediction_nb))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_nb, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

BernoulliNB()
              precision    recall  f1-score   support

           0       0.94      0.61      0.74     84509
           1       0.12      0.59      0.20      7437

    accuracy                           0.61     91946
   macro avg       0.53      0.60      0.47     91946
weighted avg       0.88      0.61      0.70     91946

[[51629 32880]
 [ 3058  4379]]
Accuracy of the model on Testing Sample Data: 0.7


# **XGBOOST(Oversampling)**

In [ ]:
# Xtreme Gradient Boosting (XGBoost)
from xgboost import XGBClassifier
clf=XGBClassifier(max_depth=2, learning_rate=0.01, n_estimators=200, objective='binary:logistic', booster='gbtree')

# Printing all the parameters of XGBoost
print(clf)

# Creating the model on Training Data
XGB=clf.fit(X_train_mlos,y_train_mlos)
prediction_xgb=XGB.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_xgb))
print(metrics.confusion_matrix(y_test_ml, prediction_xgb))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_xgb, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

XGBClassifier(learning_rate=0.01, max_depth=2, n_estimators=200)
              precision    recall  f1-score   support

           0       0.94      0.82      0.88     84509
           1       0.17      0.41      0.24      7437

    accuracy                           0.79     91946
   macro avg       0.55      0.61      0.56     91946
weighted avg       0.88      0.79      0.83     91946

[[69492 15017]
 [ 4419  3018]]
Accuracy of the model on Testing Sample Data: 0.83
